In [3]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go


Connect cassandra

In [4]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- system_schema
- system_distributed
- my_keyspace
- system
- settrade_api
- system_traces
- data_quran


Connenct Setrade API

In [4]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
                app_id="rirpANWTJCOHVgnn",                                 
                app_secret="P7iCFYB6JX0cdLXogMwU0J2yNsrpne/nzX2pFy6kiA8=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="wanneeba-D")            

account_info = deri.get_account_info()

account_info                                              

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [5]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 4400, 'totalNoSideVolume': 9700, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 1.34277, 'pe': 38.73, 'pbv': 5.98, 'percentYield': 1.5199999809265137, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [20]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
normalized=True,)

In [21]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,349695,1677603600,73.00,73.50,72.50,72.50,13912341,1.013501e+09
1,349695,1677690000,72.50,72.50,71.25,71.75,34605326,2.487860e+09
2,349695,1677776400,71.50,71.50,70.25,70.50,25367513,1.791836e+09
3,349695,1678122000,70.75,71.00,69.75,69.75,26164171,1.837748e+09
4,349695,1678208400,69.25,70.00,69.00,69.75,32814418,2.278454e+09
...,...,...,...,...,...,...,...,...
495,349695,1742403600,65.25,70.00,65.00,70.00,4405101,2.908413e+08
496,349695,1742490000,69.00,91.25,59.25,67.00,4222400,2.981925e+08
497,349695,1742749200,64.75,65.25,64.00,65.25,2254101,1.456885e+08
498,349695,1742835600,52.00,52.00,50.00,52.00,314400,1.621875e+07


In [17]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [22]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()


Create keyspace

In [23]:
keyspace_name = 'data_stock'

session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)

In [24]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- data_stock
- system_schema
- system_distributed
- my_keyspace
- system
- settrade_api
- system_traces
- data_quran


Use Keyspace

In [28]:
session.set_keyspace('data_stock')

Creat table for finalcail data

In [47]:
Creat_table_query = session.prepare("""
    INSERT INTO stock_fun_data (
        symbol, aumSize, average, change, eps, exchange,
        exercisePrice, exerciseRatio, high, impliedVolatility,
        inav, instrumentType, intrinsicValue, last, lastTradingDate,
        low, marketStatus, maturityDate, moneyness, pbv,
        pe, percentChange, percentYield, securityType, theoretical,
        toLastTrade, totalVolume, underlying, underlyingPrice
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

get data from api(datafun) Pull data

In [31]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 4400, 'totalNoSideVolume': 9700, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 1.34277, 'pe': 38.73, 'pbv': 5.98, 'percentYield': 1.5199999809265137, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [48]:
from datetime import datetime

def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except Exception:
        return None
try:
    session.execute(Creat_table_query, (
        res.get("symbol"),
        res.get("aumSize"),
        res.get("average"),
        res.get("change"),
        res.get("eps"),
        res.get("exchange"),
        res.get("exercisePrice"),
        res.get("exerciseRatio"),
        res.get("high"),
        res.get("impliedVolatility"),
        res.get("inav"),
        res.get("instrumentType"),
        res.get("intrinsicValue"),
        res.get("last"),
        to_date(res.get("lastTradingDate")),
        res.get("low"),
        res.get("marketStatus"),
        to_date(res.get("maturityDate")),
        res.get("moneyness"),
        res.get("pbv"),
        res.get("pe"),
        res.get("percentChange"),
        res.get("percentYield"),
        res.get("securityType"),
        res.get("theoretical"),
        res.get("toLastTrade"),
        res.get("totalVolume"),
        res.get("underlying"),
        res.get("underlyingPrice")
    ))
    print("✅ Inserted AOT data successfully.")
except Exception as e:
    print("❌ Insert failed:", e)

✅ Inserted AOT data successfully.


In [56]:
keyspace_name = "data_stock"
session.set_keyspace(keyspace_name)
table_name = " stock_fun_data"
rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")
df = pd.DataFrame(rows)
df.head(10) 

,symbol,aumsize,average,change,eps,exchange,exerciseprice,exerciseratio,high,impliedvolatility,...,pbv,pe,percentchange,percentyield,securitytype,theoretical,tolasttrade,totalvolume,underlying,underlyingprice
0,AOT,None,None,None,1.34277,None,None,None,None,None,...,5.98,38.73,None,1.52,CS,None,None,0,None,None


In [58]:

# ✅ ฟังก์ชันแปลงวันที่ string → datetime.date
def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except:
        return None

# ✅ ฟังก์ชัน insert ข้อมูลแต่ละหุ้น
def insert_financial_data(symbol):
    try:
        print(f"🔍 ดึงข้อมูล {symbol}...")

        res = market.get_quote_symbol(symbol)

        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}.")
            return

        # เตรียมข้อมูล
        columns = [
            "symbol", "aumSize", "average", "change", "eps", "exchange",
            "exercisePrice", "exerciseRatio", "high", "impliedVolatility",
            "inav", "instrumentType", "intrinsicValue", "last", "lastTradingDate",
            "low", "marketStatus", "maturityDate", "moneyness", "pbv",
            "pe", "percentChange", "percentYield", "securityType", "theoretical",
            "toLastTrade", "totalVolume", "underlying", "underlyingPrice"
        ]

        values = [res.get(col) for col in columns]
        values[14] = to_date(values[14])  # lastTradingDate
        values[17] = to_date(values[17])  # maturityDate

        insert_query = session.prepare(f"""
            INSERT INTO stock_fun_data ({", ".join(columns)})
            VALUES ({", ".join(["?"] * len(columns))})
        """)

        session.execute(insert_query, values)
        print(f"✅ Insert สำเร็จ: {symbol}")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับ {symbol}: {e}")

# ✅ โหลดรายชื่อหุ้น
try:
    symbols_df = pd.read_excel("get_stcok_name/stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ เริ่ม loop insert
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(2)  # ควบคุมไม่ให้เรียก API รัวเกินไป

🔍 ดึงข้อมูล 24CS...
✅ Insert สำเร็จ: 24CS
🔍 ดึงข้อมูล 2S...
✅ Insert สำเร็จ: 2S
🔍 ดึงข้อมูล 3BBIF...
✅ Insert สำเร็จ: 3BBIF
🔍 ดึงข้อมูล A...
✅ Insert สำเร็จ: A
🔍 ดึงข้อมูล A5...
✅ Insert สำเร็จ: A5
🔍 ดึงข้อมูล AAI...
✅ Insert สำเร็จ: AAI
🔍 ดึงข้อมูล AAV...
✅ Insert สำเร็จ: AAV
🔍 ดึงข้อมูล ABM...
✅ Insert สำเร็จ: ABM
🔍 ดึงข้อมูล ACAP...
✅ Insert สำเร็จ: ACAP
🔍 ดึงข้อมูล ACC...
✅ Insert สำเร็จ: ACC
🔍 ดึงข้อมูล ACE...
✅ Insert สำเร็จ: ACE
🔍 ดึงข้อมูล ACG...
✅ Insert สำเร็จ: ACG
🔍 ดึงข้อมูล ADB...
✅ Insert สำเร็จ: ADB
🔍 ดึงข้อมูล ADD...
✅ Insert สำเร็จ: ADD
🔍 ดึงข้อมูล ADVANC...
✅ Insert สำเร็จ: ADVANC
🔍 ดึงข้อมูล ADVICE...
✅ Insert สำเร็จ: ADVICE
🔍 ดึงข้อมูล AE...
✅ Insert สำเร็จ: AE
🔍 ดึงข้อมูล AEONTS...
✅ Insert สำเร็จ: AEONTS
🔍 ดึงข้อมูล AF...
✅ Insert สำเร็จ: AF
🔍 ดึงข้อมูล AFC...
✅ Insert สำเร็จ: AFC
🔍 ดึงข้อมูล AGE...
✅ Insert สำเร็จ: AGE
🔍 ดึงข้อมูล AH...
✅ Insert สำเร็จ: AH
🔍 ดึงข้อมูล AHC...
✅ Insert สำเร็จ: AHC
🔍 ดึงข้อมูล AI...
✅ Insert สำเร็จ: AI
🔍 ดึงข้อมูล AIE...
✅ Insert สำ

Show data_fan

In [5]:
keyspace_name = "data_stock"
session.set_keyspace(keyspace_name)
table_name = " stock_fun_data"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape


(917, 29)

In [6]:
df.head()

,symbol,aumsize,average,change,eps,exchange,exerciseprice,exerciseratio,high,impliedvolatility,...,pbv,pe,percentchange,percentyield,securitytype,theoretical,tolasttrade,totalvolume,underlying,underlyingprice
0,PPPM,None,None,None,-0.18924,None,None,None,None,None,...,0.67,0.00,None,0.00,CS,None,None,0,None,None
1,TPCH,None,None,None,0.63348,None,None,None,None,None,...,0.50,4.57,None,9.76,CS,None,None,0,None,None
2,KPNREIT,None,None,None,NaN,None,None,None,None,None,...,NaN,NaN,None,0.00,CS,None,None,0,None,None
3,POLY,None,None,None,0.33433,None,None,None,None,None,...,3.05,15.31,None,4.55,CS,None,None,0,None,None
4,QHBREIT,None,None,None,NaN,None,None,None,None,None,...,NaN,NaN,None,0.00,CS,None,None,0,None,None


In [8]:
rows_symbol = session.execute(f"SELECT * FROM data_stock.stock_fun_data WHERE symbol = 'ZIGA'")
df_symbol = pd.DataFrame(rows_symbol)
df_symbol.head()

,symbol,aumsize,average,change,eps,exchange,exerciseprice,exerciseratio,high,impliedvolatility,...,pbv,pe,percentchange,percentyield,securitytype,theoretical,tolasttrade,totalvolume,underlying,underlyingprice
0,ZIGA,None,None,None,0.02547,None,None,None,None,None,...,1.33,34.63,None,0.0,CS,None,None,0,None,None


In [8]:
row = df_symbol.iloc[0]
non_null = row[row.notnull()]
print(non_null)


symbol               24CS
eps              -0.57629
instrumenttype      STOCK
marketstatus        Close
pbv                  2.59
pe                    0.0
percentyield          0.0
securitytype           CS
totalvolume             0
Name: 0, dtype: object


In [59]:
# ดึงข้อมูลเฉพาะคอลัมน์ที่ต้องการจาก Cassandra
rows_symbol_nonnull = session.execute(
    """
    SELECT symbol, eps, instrumenttype, marketstatus, pbv, pe, percentyield, securitytype, totalvolume
    FROM data_stock.stock_fun_data
    """
)
# แปลงเป็น DataFrame
df_symbol_nonnull = pd.DataFrame(rows_symbol_nonnull)
# แสดงผลลัพธ์
df_symbol_nonnull.head()

,symbol,eps,instrumenttype,marketstatus,pbv,pe,percentyield,securitytype,totalvolume
0,PPPM,-0.18924,STOCK,Close,0.67,0.00,0.00,CS,0
1,TPCH,0.63348,STOCK,Close,0.50,4.57,9.76,CS,0
2,KPNREIT,NaN,STOCK,Close,NaN,NaN,0.00,CS,0
3,POLY,0.33433,STOCK,Close,3.05,15.31,4.55,CS,0
4,QHBREIT,NaN,STOCK,Close,NaN,NaN,0.00,CS,0


In [ ]:
keyspace_name = "data_stock"
session.set_keyspace(keyspace_name)
table_name = " stock_fun_data"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape